In [496]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from textblob import Word 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package stopwords to /Users/rand/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/rand/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/rand/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [497]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [498]:
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [499]:
test.head(10)

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...
7,22,NaN,NaN,Hey! How are you?
8,27,NaN,NaN,What a nice hat?
9,29,NaN,NaN,Fuck off!


## Data Preprocessing

In [500]:
def lable_df_func(dataset, flag):

    if flag == 0:
        lable_list = [TextBlob(i) for i in dataset['text']]
        lable_values = [[lable.sentiment.polarity] for lable in lable_list]
        lable_df = pd.DataFrame(lable_values, columns=["polarity"])
    if flag == 1:
        lable_list = [TextBlob(str(i)) for i in dataset]
        lable_values = [[lable.sentiment.polarity, str(lable)] for lable in lable_list]
        lable_df = pd.DataFrame(lable_values, columns=["polarity_lem", "lems"])
    
    return lable_df


In [501]:
def set_weight(data, lable_df, flag):
    if flag == 0:
        data = pd.concat([data, lable_df], axis=1)
    else:
        data = pd.concat([data, lable_df["lems"]], axis=1)
    return data


In [502]:
def preprocess(dataset):
    golden_res = []
    stop_words = set(stopwords.words('english'))
    for key in dataset["text"]:
        occupy_list = word_tokenize(key) 
        occupy_list=[word.lower() for word in occupy_list if word.isalpha()]
        lable_cata = [i for i in occupy_list if not i in stop_words] 
        lable_cata = [] 

        for key in occupy_list: 
            if key not in stop_words : 
                if  key != 'http':
                    lable_cata.append(key) 

        Stem_words = []
        ps = PorterStemmer()
        for key in lable_cata:
            rootWord = ps.stem(key)
            Stem_words.append(rootWord)
        lem = []
        for key in lable_cata:
            lable1 = Word(key).lemmatize("n")
            lable2 = Word(lable1).lemmatize("v")
            lable3 = Word(lable2).lemmatize("a")
            lem.append(Word(lable3).lemmatize())
        golden_res.append(lem)
    return golden_res


In [503]:
def replace(dataset):
    dataset["lems"]= dataset["lems"].str.replace("[", "") 
    dataset["lems"]= dataset["lems"].str.replace("]", "") 
    dataset["lems"]= dataset["lems"].str.replace("\'", "") 
    dataset["lems"]= dataset["lems"].str.replace(",", " ")
    

In [504]:
def run_all_func(train, test):
    # train function
    lable_df_train = lable_df_func(train, 0)
    train_ret = set_weight(train, lable_df_train, 0)
    golden_res_train = preprocess(train_ret)
    lable_df_train = lable_df_func(golden_res_train, 1)
    train_ret = set_weight(train_ret, lable_df_train, 1)
    replace(train_ret)
    
    # test function
    golden_res_test = preprocess(test)
    lable_df_test = lable_df_func(golden_res_test, 1)
    test_ret = set_weight(test, lable_df_test, 0)
    replace(test_ret)
    
    return train_ret, test_ret
    
train, test = run_all_func(train, test)


In [508]:
test.head()  


,id,keyword,location,text,polarity_lem,lems
0,0,NaN,NaN,Just happened a terrible car crash,-1.00,happen terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",0.25,hear earthquake different city stay safe ...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",0.00,forest fire spot pond goose flee across ...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,0.40,apocalypse light spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0.00,typhoon soudelor kill china taiwan


In [507]:
train.head()  


,id,keyword,location,text,target,polarity,lems
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.00000,deed reason earthquake may allah forgive u
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.10000,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,-0.01875,resident ask place notify officer evacuat...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.00000,people receive wildfire evacuation order ...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.00000,get send photo ruby alaska smoke wildfir...


## build model

In [510]:
count_model = CountVectorizer()
X_train = count_model.fit_transform(train["lems"])
X_test = count_model.transform(test["lems"])


In [511]:
tfidf_model = TfidfVectorizer()
X_train_tfidf = tfidf_model.fit_transform(train["lems"])
X_test_tfidf = tfidf_model.transform(test["lems"])


In [517]:
bayes_model = MultinomialNB()
scores = model_selection.cross_val_score(bayes_model, X_train, train["target"], cv=3, scoring="f1")
bayes_model.fit( X_train, train["target"])
bayes_model_tfidf = MultinomialNB()
scores = model_selection.cross_val_score(bayes_model_tfidf, X_train_tfidf, train["target"], cv=3, scoring="f1")
bayes_model_tfidf.fit(X_train_tfidf, train["target"])


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

## Generate submission file

In [518]:
def save_csv():
    sample_submission = pd.read_csv("sample_submission.csv")
    sample_submission["target"]= bayes_model.predict(X_test)
    sample_submission.to_csv("submission.csv", index=False)


In [519]:
save_csv()